In [9]:
import numpy as np
import pandas as pd
import os
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from scipy.interpolate import UnivariateSpline
import statsmodels.api as sm
import matplotlib.pyplot as plt
import collections
import eli5
from eli5.sklearn import PermutationImportance
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_column", 999)
#print(os.listdir("../input"))

## 1. Import Data

In [6]:
tourney_results = pd.read_csv('MDataFiles_Stage1/MNCAATourneyDetailedResults.csv')
seeds = pd.read_csv('MDataFiles_Stage1/MNCAATourneySeeds.csv')
regular_results = pd.read_csv('MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')

regular_results['WEFFG'] = regular_results['WFGM'] / regular_results['WFGA']
regular_results['WEFFG3'] = regular_results['WFGM3'] / regular_results['WFGA3']
regular_results['WDARE'] = regular_results['WFGM3'] / regular_results['WFGM']
regular_results['WTOQUETOQUE'] = regular_results['WAst'] / regular_results['WFGM']

regular_results['LEFFG'] = regular_results['LFGM'] / regular_results['LFGA']
regular_results['LEFFG3'] = regular_results['LFGM3'] / regular_results['LFGA3']
regular_results['LDARE'] = regular_results['LFGM3'] / regular_results['LFGM']
regular_results['LTOQUETOQUE'] = regular_results['LAst'] / regular_results['LFGM']
regular_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,WEFFG,WEFFG3,WDARE,WTOQUETOQUE,LEFFG,LEFFG3,LDARE,LTOQUETOQUE
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20,0.465517,0.214286,0.111111,0.481481,0.415094,0.200000,0.090909,0.363636
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16,0.419355,0.400000,0.307692,0.615385,0.358209,0.250000,0.250000,0.291667
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23,0.413793,0.444444,0.333333,0.625000,0.301370,0.115385,0.136364,0.409091
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23,0.473684,0.333333,0.166667,0.611111,0.367347,0.272727,0.333333,0.500000
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14,0.491803,0.428571,0.200000,0.400000,0.387097,0.375000,0.250000,0.500000


In [8]:
regular_results.shape

(100423, 42)

In [10]:
regular_results.T

0         1         2         3         4         5       \
Season           2003      2003      2003      2003      2003      2003   
DayNum             10        10        11        11        11        11   
WTeamID          1104      1272      1266      1296      1400      1458   
WScore             68        70        73        56        77        81   
LTeamID          1328      1393      1437      1457      1208      1186   
LScore             62        63        61        50        71        55   
WLoc                N         N         N         N         N         H   
NumOT               0         0         0         0         0         0   
WFGM               27        26        24        18        30        26   
WFGA               58        62        58        38        61        57   
WFGM3               3         8         8         3         6         6   
WFGA3              14        20        18         9        14        12   
WFTM               11        10        17        17        11        23   
WFTA               18        19        29        31        13        27   
WOR                14        15        17         6        17        12   
WDR                24        28        26        19        22        24   
WAst               13        16        15        11        12        12   
WTO                23        13        10        12        14         9   
WStl                7         4         5        14         4         9   
WBlk                1         4         2         2         4         3   
WPF                22        18        25        18        20        18   
LFGM               22        24        22        18        24        20   
LFGA               53        67        73        49        62        46   
LFGM3               2         6         3         6         6         3   
LFGA3              10        24        26        22        16        11   
LFTM               16         9        14         8        17        12   
LFTA               22        20        23        15        27        17   
LOR                10        20        31        17        21         6   
LDR                22        25        22        20        15        22   
LAst                8         7         9         9        12         8   
LTO                18        12        12        19        10        19   
LStl                9         8         2         4         7         4   
LBlk                2         6         5         3         1         3   
LPF                20        16        23        23        14        25   
WEFFG        0.465517  0.419355  0.413793  0.473684  0.491803   0.45614   
WEFFG3       0.214286       0.4  0.444444  0.333333  0.428571       0.5   
WDARE        0.111111  0.307692  0.333333  0.166667       0.2  0.230769   
WTOQUETOQUE  0.481481  0.615385     0.625  0.611111       0.4  0.461538   
LEFFG        0.415094  0.358209   0.30137  0.367347  0.387097  0.434783   
LEFFG3            0.2      0.25  0.115385  0.272727     0.375  0.272727   
LDARE        0.090909      0.25  0.136364  0.333333      0.25      0.15   
LTOQUETOQUE  0.363636  0.291667  0.409091       0.5       0.5       0.4   

               6         7         8         9         10        11      \
Season           2003      2003      2003      2003      2003      2003   
DayNum             12        12        12        12        13        13   
WTeamID          1161      1186      1194      1458      1166      1202   
WScore             80        75        71        84       106        74   
LTeamID          1236      1457      1156      1296      1426      1106   
LScore             62        61        66        56        50        73   
WLoc                H         N         N         H         H         N   
NumOT               0         0         0         0         0         0   
WFGM               23        28        28        32        41        29   
WFGA               55        62        

In [11]:
def prepare_data(df):
    dfswap = df[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT', 
    'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]

    dfswap.loc[df['WLoc'] == 'H', 'WLoc'] = 'A'
    dfswap.loc[df['WLoc'] == 'A', 'WLoc'] = 'H'
    df.columns.values[6] = 'location'
    dfswap.columns.values[6] = 'location'    
      
    df.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(df.columns)]
    dfswap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(dfswap.columns)]

    output = pd.concat([df, dfswap]).reset_index(drop=True)
    output.loc[output.location=='N','location'] = '0'
    output.loc[output.location=='H','location'] = '1'
    output.loc[output.location=='A','location'] = '-1'
    output.location = output.location.astype(int)
    
    output['PointDiff'] = output['T1_Score'] - output['T2_Score']
    
    return output

In [12]:
regular_data = prepare_data(regular_results)
tourney_data = prepare_data(tourney_results)

In [16]:
regular_data.head()

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,T1_FGM3,T1_FGA3,T1_FTM,T1_FTA,T1_OR,T1_DR,T1_Ast,T1_TO,T1_Stl,T1_Blk,T1_PF,T2_FGM,T2_FGA,T2_FGM3,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF,T1_EFFG,T1_EFFG3,T1_DARE,T1_TOQUETOQUE,T2_EFFG,T2_EFFG3,T2_DARE,T2_TOQUETOQUE,PointDiff
0,2003,10,1104,68,1328,62,0,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20,0.465517,0.214286,0.111111,0.481481,0.415094,0.200000,0.090909,0.363636,6
1,2003,10,1272,70,1393,63,0,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16,0.419355,0.400000,0.307692,0.615385,0.358209,0.250000,0.250000,0.291667,7
2,2003,11,1266,73,1437,61,0,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23,0.413793,0.444444,0.333333,0.625000,0.301370,0.115385,0.136364,0.409091,12
3,2003,11,1296,56,1457,50,0,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23,0.473684,0.333333,0.166667,0.611111,0.367347,0.272727,0.333333,0.500000,6
4,2003,11,1400,77,1208,71,0,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14,0.491803,0.428571,0.200000,0.400000,0.387097,0.375000,0.250000,0.500000,6


## 2. Feature Engineering

In [17]:
boxscore_cols = ['T1_Score', 'T2_Score', 
        'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_FTM', 'T1_FTA', 'T1_OR', 'T1_DR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_Blk', 'T1_PF', 
        'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_FTM', 'T2_FTA', 'T2_OR', 'T2_DR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk', 'T2_PF', 
        'PointDiff']

boxscore_cols = [
        'T1_FGM', 'T1_FGA', 'T1_OR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_PF', 'T1_FTM', 'T2_FTM', 'T2_FGM', 'T2_FGA', 
        'T2_OR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk', 'T1_Score', 'T2_Score', 'PointDiff',
        'T1_EFFG', 'T1_EFFG3', 'T1_DARE', 'T1_TOQUETOQUE', 'T2_EFFG', 'T2_EFFG3', 'T2_DARE', 'T2_TOQUETOQUE']
# After my analysis
#boxscore_cols = ['PointDiff', 'T1_Blk', 'T2_Blk', 'T1_Ast', 'T2_Ast', 'T1_Stl', 'T2_Stl', 'T1_FGA', 
#                 'T2_FGA', 'T1_FGM', 'T2_FGM', 'T1_DR', 'T2_DR', 'T1_Score', 'T2_Score']


# Choose a function to aggregate
funcs = [np.mean]

In [18]:
season_statistics = regular_data.groupby(["Season", 'T1_TeamID'])[boxscore_cols].agg(funcs).reset_index()
season_statistics.columns = [''.join(col).strip() for col in season_statistics.columns.values]
#Make two copies of the data
season_statistics_T1 = season_statistics.copy()
season_statistics_T2 = season_statistics.copy()

season_statistics_T1.columns = ["T1_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T1.columns)]
season_statistics_T2.columns = ["T2_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T2.columns)]
season_statistics_T1.columns.values[0] = "Season"
season_statistics_T2.columns.values[0] = "Season"

In [19]:
tourney_data = tourney_data[['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID' ,'T2_Score']]
tourney_data.head()

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score
0,2003,134,1421,92,1411,84
1,2003,136,1112,80,1436,51
2,2003,136,1113,84,1272,71
3,2003,136,1141,79,1166,73
4,2003,136,1143,76,1301,74


In [20]:
tourney_data = pd.merge(tourney_data, season_statistics_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, season_statistics_T2, on = ['Season', 'T2_TeamID'], how = 'left')
# Notice that there are Team 1 statistics, team 1 opponent's statistics, team 2 statistics and team 2 opponent statistics
tourney_data.head()

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,T1_FGMmean,T1_FGAmean,T1_ORmean,T1_Astmean,T1_TOmean,T1_Stlmean,T1_PFmean,T1_FTMmean,T1_opponent_FTMmean,T1_opponent_FGMmean,T1_opponent_FGAmean,T1_opponent_ORmean,T1_opponent_Astmean,T1_opponent_TOmean,T1_opponent_Stlmean,T1_opponent_Blkmean,T1_Scoremean,T1_opponent_Scoremean,T1_PointDiffmean,T1_EFFGmean,T1_EFFG3mean,T1_DAREmean,T1_TOQUETOQUEmean,T1_opponent_EFFGmean,T1_opponent_EFFG3mean,T1_opponent_DAREmean,T1_opponent_TOQUETOQUEmean,T2_FGMmean,T2_FGAmean,T2_ORmean,T2_Astmean,T2_TOmean,T2_Stlmean,T2_PFmean,T2_FTMmean,T2_opponent_FTMmean,T2_opponent_FGMmean,T2_opponent_FGAmean,T2_opponent_ORmean,T2_opponent_Astmean,T2_opponent_TOmean,T2_opponent_Stlmean,T2_opponent_Blkmean,T2_Scoremean,T2_opponent_Scoremean,T2_PointDiffmean,T2_EFFGmean,T2_EFFG3mean,T2_DAREmean,T2_TOQUETOQUEmean,T2_opponent_EFFGmean,T2_opponent_EFFG3mean,T2_opponent_DAREmean,T2_opponent_TOQUETOQUEmean
0,2003,134,1421,92,1411,84,24.379310,56.793103,12.275862,13.034483,16.206897,7.068966,19.103448,15.965517,15.241379,27.793103,60.965517,13.724138,15.862069,12.827586,8.827586,4.241379,71.206897,78.448276,-7.241379,0.474169,0.401956,0.252522,0.601102,0.426596,0.331984,0.293895,0.530332,24.733333,55.266667,13.166667,14.200000,15.233333,6.433333,18.300000,17.400000,11.966667,25.666667,60.400000,11.933333,13.766667,14.333333,8.000000,2.600000,72.800000,70.833333,1.966667,0.477161,0.344890,0.237292,0.592952,0.388919,0.277613,0.260372,0.502632
1,2003,136,1112,80,1436,51,30.321429,65.714286,15.178571,17.642857,14.785714,8.464286,17.750000,17.535714,11.214286,26.357143,64.678571,13.107143,15.464286,16.857143,5.964286,2.392857,85.214286,70.250000,14.964286,0.473342,0.362485,0.230786,0.595322,0.403696,0.304470,0.240722,0.583080,24.827586,55.862069,12.965517,14.206897,14.068966,6.862069,15.896552,12.862069,10.551724,22.758621,55.068966,9.586207,13.275862,13.000000,7.103448,3.655172,67.793103,63.137931,4.655172,0.471687,0.374807,0.213022,0.597435,0.399587,0.318975,0.319844,0.544214
2,2003,136,1113,84,1272,71,27.206897,56.896552,13.689655,15.551724,14.000000,5.206897,19.413793,17.551724,14.137931,24.793103,55.655172,11.000000,14.068966,15.517241,6.000000,3.931034,75.965517,69.172414,6.793103,0.522554,0.331964,0.120461,0.582106,0.411585,0.315173,0.213399,0.557069,26.275862,60.000000,14.068966,16.620690,13.793103,7.379310,18.758621,14.965517,13.379310,23.275862,57.862069,12.344828,13.310345,15.068966,7.275862,3.172414,74.517241,65.827586,8.689655,0.449325,0.353201,0.268420,0.657300,0.390519,0.310019,0.264035,0.580375
3,2003,136,1141,79,1166,73,26.620690,52.689655,10.586207,15.620690,18.241379,7.103448,20.965517,19.275862,15.586207,26.344828,57.931034,12.241379,11.793103,16.068966,8.448276,2.482759,79.344828,73.241379,6.103448,0.531399,0.404501,0.260579,0.598454,0.444717,0.356589,0.200137,0.423271,28.696970,57.454545,10.878788,16.818182,13.363636,8.393939,17.272727,13.878788,11.696970,23.878788,55.333333,11.060606,12.363636,17.060606,6.333333,2.575758,79.242424,64.333333,14.909091,0.507076,0.391517,0.276577,0.597444,0.423279,0.343467,0.205701,0.512530
4,2003,136,1143,76,1301,74,27.344828,58.724138,11.241379,16.000000,14.172414,6.551724,17.103448,13.379310,11.896552,25.379310,58.793103,11.172414,16.034483,14.931034,5.931034,2.517241,74.482759,69.758621,4.724138,0.487332,0.382172,0.224321,0.605668,0.418497,0.305100,0.286564,0.640571,24.333333,53.333333,9.733333,14.666667,14.200000,7.766667,18.666667,15.766667,15.400000,23.433333,53.133333,10.533333,12.566667,14.633333,7.433333,2.833333,72.400000,68.000000,4.400000,0.488343,0.397771,0.349052,0.639612,0.416862,0.304249,0.262301,0.525119


In [22]:
# Extract the teams that make it to the tournament and see how they do with respect to the others
regular_season_effects = regular_data[['Season','T1_TeamID','T2_TeamID','PointDiff']].copy()
regular_season_effects['T1_TeamID'] = regular_season_effects['T1_TeamID'].astype(str)
regular_season_effects['T2_TeamID'] = regular_season_effects['T2_TeamID'].astype(str)
regular_season_effects['win'] = np.where(regular_season_effects['PointDiff']>0,1,0)
march_madness = pd.merge(seeds[['Season','TeamID']],seeds[['Season','TeamID']],on='Season')
march_madness.columns = ['Season', 'T1_TeamID', 'T2_TeamID']
march_madness.T1_TeamID = march_madness.T1_TeamID.astype(str)
march_madness.T2_TeamID = march_madness.T2_TeamID.astype(str)
regular_season_effects = pd.merge(regular_season_effects, march_madness, on = ['Season','T1_TeamID','T2_TeamID'])
regular_season_effects.shape

(10462, 5)

In [23]:
regular_season_effects

,Season,T1_TeamID,T2_TeamID,PointDiff,win
0,2003,1104,1328,6,1
1,2003,1272,1393,7,1
2,2003,1323,1237,44,1
3,2003,1242,1221,24,1
4,2003,1390,1462,1,1
...,...,...,...,...,...
10457,2021,1458,1345,-4,0
10458,2021,1196,1281,-2,0
10459,2021,1417,1332,-8,0
10460,2021,1281,1261,-6,0


### 2.2 Team Quality

In [25]:
def normalize_column(values):
  themean = np.mean(values)
  thestd = np.std(values)
  norm = (values - themean)/(thestd) 
  return(pd.DataFrame(norm))

def team_quality(season):
    formula = 'win~-1+T1_TeamID+T2_TeamID'
    glm = sm.GLM.from_formula(formula=formula, 
                              data=regular_season_effects.loc[regular_season_effects.Season==season,:], 
                              family=sm.families.Binomial()).fit()
    quality = pd.DataFrame(glm.params).reset_index()
    quality.columns = ['TeamID','quality']
    quality['Season'] = season
    quality['quality'] = normalize_column(quality['quality'])
    quality['quality'] = np.exp(quality['quality'])
    quality = quality.loc[quality.TeamID.str.contains('T1_')].reset_index(drop=True)
    quality['TeamID'] = quality['TeamID'].apply(lambda x: x[10:14]).astype(int)
    print(quality['quality'].mean(), quality['quality'].std())
    return quality

In [26]:
# This is metric to measure the team's strength, in this case, this is a logistic regression and we
# the coefficients
glm_quality = pd.concat([team_quality(2003),
                         team_quality(2004),
                         team_quality(2005),
                         team_quality(2006),
                         team_quality(2007),
                         team_quality(2008),
                         team_quality(2009),
                         team_quality(2010),
                         team_quality(2011),
                         team_quality(2012),
                         team_quality(2013),
                         team_quality(2014),
                         team_quality(2015),
                         team_quality(2016),
                         team_quality(2017),
                         team_quality(2018),
                         team_quality(2019),
                         team_quality(2021)]).reset_index(drop=True)

0.8054096184989878 0.34029540011404863
0.8152913116376672 0.47589866838662864
1.0137586551206133 0.905946244559899
2.4627680988700624 0.9114114356248308
2.5869072580747225 0.8489693853991197
1.0238605730130454 0.4021318919988505
2.656134846964437 0.7336158250228532
2.465594657047584 1.143832477945101
2.5337249098260015 0.8277680981908614
0.8979844549778124 0.582639783188835
0.9243847418430284 0.38339610378942673
2.536262608189653 1.161365145312731
2.5318181465950333 1.998633271240072
0.8150475572661224 0.37265139356991
0.9951121907243218 1.6762242340496263
0.7509430640146026 0.6320569991850188
2.3792422910106685 1.2160062651934123
2.610071166323622 1.306759675341535
